### Mideapipe를 사용하기 위한 가상환경 생성하기

In [ ]:
# conda create -n mediapipe-env python=3.9 -y
# conda activate mediapipe-env

# pip install opencv-python mediapipe protobuf==3.20.3

## 얼굴 감지 기능
- 얼굴 위치와 크기 탐지 (+ 코끝, 눈 위치 등 keypoint 일부) 
- mp_face_detection.FaceDetection(...)        
https://www.pexels.com/ko-kr/video/3256542/  대화나누는 동영상

In [2]:
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴 특징을 그리기 위한 drawing_tuils 모듈 사용

# 웹캠 열기
cap = cv2.VideoCapture(0)

# FaceDetection 솔루션 초기화 (실시간 모드로 사용)
with mp_face_detection.FaceDetection(
    # 모델의 선택(model_selection)과 최소 감지 신뢰도(min_detection_confidence) 설정
    model_selection=0, 
    min_detection_confidence=0.5
) as face_detection:
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: 
            break

        # BGR → RGB로 변환 (MediaPipe는 RGB 이미지 필요)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
        image.flags.writeable = False  # 성능 최적화 (읽기 전용)
        
        # 얼굴 감지 실행
        results = face_detection.process(image)

        image.flags.writeable = True  # 다시 쓰기 가능 상태로 변경
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #  얼굴 감지 결과가 있을 경우 실행
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)
                
        # 결과 이미지 출력 (크기 축소)
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None , fx=0.5, fy=0.5))
        
        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'mediapipe'

## 전신 포즈 추정
- MediaPipe의 Pose 모듈은 사람의 전신 33개 주요 관절(keypoints)을 실시간으로 감지 : 자세 분석
    - 얼굴: nose, eyes, ears
    - 상체: shoulders, elbows, wrists
    - 하체: hips, knees, ankles
    - 발끝, 발뒤꿈치 등
- mp_pose.Pose(...)

In [2]:
import cv2
import mediapipe as mp

# MediaPipe의 pose 솔루션과 drawing 유틸리티 불러오기
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# 웹캠 열기
cap = cv2.VideoCapture(0)

# pose 솔루션 초기화 (실시간 모드로 사용)
with mp_pose.Pose(
    static_image_mode=False,        # 동영상(실시간) 모드
    model_complexity=1,             # 모델 복잡도 (0~2)
    enable_segmentation=False,      # 사람 영역 분할 안 함
    min_detection_confidence=0.5    # 최소 감지 신뢰도
) as pose:
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # BGR → RGB로 변환 (MediaPipe는 RGB 이미지 필요)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False  # 성능 최적화 (읽기 전용)

        # 포즈 추정 실행
        results = pose.process(image)

        image.flags.writeable = True  # 다시 쓰기 가능 상태로 변경
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 포즈 추정 결과가 있을 경우 랜드마크 그리기
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image,                         # 원본 이미지
                results.pose_landmarks,        # 감지된 포즈 랜드마크
                mp_pose.POSE_CONNECTIONS       # 관절 연결선
            )

        # 결과 이미지 출력 (크기 축소)
        cv2.imshow('MediaPipe Pose Detection', cv2.resize(image, None, fx=0.7, fy=0.7))
        
        # ESC 키(27번) 누르면 종료
        if cv2.waitKey(1) == ord('q'):
            break

# 웹캠과 윈도우 닫기
cap.release()
cv2.destroyAllWindows()


## MediaPipe Hands: 손 관절 21개 감지
- 21개 손가락 관절 위치 추정(제스처 인식 등)
- mp_hands.Hands(...)

In [3]:
import cv2
import mediapipe as mp

# MediaPipe 손 추적 모듈과 그리기 유틸리티 불러오기
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# 웹캠 켜기
cap = cv2.VideoCapture(0)

# 손 추적기 초기화
with mp_hands.Hands(
    max_num_hands=2,                   # 최대 추적 손 개수
    min_detection_confidence=0.5,      # 감지 신뢰도
    min_tracking_confidence=0.5        # 추적 신뢰도
) as hands:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("카메라 프레임을 읽을 수 없습니다.")
            continue

        # BGR → RGB 변환 (MediaPipe는 RGB 입력 필요)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False  # 성능 향상을 위한 설정

        # 손 감지 수행
        results = hands.process(image)

        # 다시 BGR로 변환 후 쓰기 가능 상태로 전환
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 감지된 손이 있으면 각 손의 랜드마크를 그림
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image, 
                    hand_landmarks, 
                    mp_hands.HAND_CONNECTIONS)

        # 결과 출력
        cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
        if cv2.waitKey(1) == ord('q') :  # ESC 키로 종료
            break

cap.release()
cv2.destroyAllWindows()
